In [121]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter

In [122]:
stocks = pd.read_csv('/Users/hashimkhan/Desktop/algo_trading_final/sp_500_stocks.csv')
from secret_case import API_TOKEN

In [123]:
#Function sources from: https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
def chunks (lst, n):
    """Yield succisve n-sized chunks form lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [124]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
dataframes = []

for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data = rq.get(batch_api_url, headers=headers, params=querystring).json()

    rows = []
    for result in data.get('quoteResponse', {}).get('result', []):
        symbol = result.get('symbol', np.nan)
        
        year_high_percent = float(result.get('fiftyTwoWeekLowChangePercent', np.nan))
        year_low_percent = float(result.get('fiftyTwoWeekHighChangePercent', np.nan))

        year_price_return = ((year_high_percent + year_low_percent) / 2)

        regular_market_price = float(result.get('regularMarketPrice', np.nan))

        rows.append([symbol, regular_market_price, year_price_return, np.nan])

    dataframe = pd.DataFrame(rows, columns=my_columns)
    dataframes.append(dataframe)

final_dataframe = pd.concat(dataframes, ignore_index=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,110.50,-0.148367,NaN
1,AAL,12.61,-0.128348,NaN
2,AAP,54.82,-0.356030,NaN
3,AAPL,170.43,0.116156,NaN
4,ABBV,153.13,0.040090,NaN
...,...,...,...,...
499,YUM,121.98,0.012371,NaN
500,ZBH,110.76,-0.089179,NaN
501,ZBRA,225.91,-0.171387,NaN
502,ZION,33.17,0.203902,NaN


In [125]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.head()

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,345,NVDA,424.68,1.386180,NaN
1,194,FTI,21.29,0.810315,NaN
2,390,RCL,91.96,0.652478,NaN
3,198,GE,110.89,0.617819,NaN
4,84,CCL,13.98,0.501571,NaN


In [126]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print("That is not a number! \nPlease try again:")
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

10000000


In [127]:
position_size = float(portfolio_size) /len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe.head()

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,345,NVDA,424.68,1.386180,470.0
1,194,FTI,21.29,0.810315,9394.0
2,390,RCL,91.96,0.652478,2174.0
3,198,GE,110.89,0.617819,1803.0
4,84,CCL,13.98,0.501571,14306.0


## Below Contains a More Realistic Momentum Strategy

In [128]:
hqm_columns = [
    'Ticker',
    'Price',
    'One-Year Price Return',
    'One-Year Return Percentile',
    '200-Day Price Return',
    '200-Day Return Percentile',
    '50-Day Price Return',
    '50-Day Return Percentile',
    'HQM Score'
]

In [129]:
dataframes1 = []


for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    rows1 = []
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data1 = rq.get(batch_api_url, headers=headers, params=querystring).json()

    for result in data1.get('quoteResponse', {}).get('result', []):
        
        symbol = result.get('symbol', np.nan)
       
        year_high_percent = float(result.get('fiftyTwoWeekLowChangePercent', np.nan))
        year_low_percent = float(result.get('fiftyTwoWeekHighChangePercent', np.nan))

        year_price_return = ((year_high_percent + year_low_percent) / 2)

        regular_market_price = float(result.get('regularMarketPrice', np.nan))
        day200_return = float(result.get('twoHundredDayAverageChangePercent', np.nan))
        day50_return = float(result.get('fiftyDayAverageChangePercent', np.nan))

        rows1.append([symbol, regular_market_price, year_price_return, 0, day200_return, 0, day50_return, 0, 0])

    dataframe = pd.DataFrame(rows1, columns=hqm_columns)
    dataframes1.append(dataframe)

hqm_dataframe = pd.concat(dataframes1, ignore_index=True)

In [130]:
time_periods = ['One-Year', '200-Day', '50-Day']

for time_period in time_periods:
    hqm_dataframe[f'{time_period} Return Percentile'] = hqm_dataframe[f'{time_period} Price Return'].rank(pct=True) * 100

hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,200-Day Price Return,200-Day Return Percentile,50-Day Price Return,50-Day Return Percentile,HQM Score
0,A,110.50,-0.148367,9.850107,-0.167640,12.633833,-0.081727,27.623126,0
1,AAL,12.61,-0.128348,14.775161,-0.165346,13.062099,-0.159468,3.640257,0
2,AAP,54.82,-0.356030,0.214133,-0.485734,0.214133,-0.178987,1.927195,0
3,AAPL,170.43,0.116156,71.092077,0.027022,68.308351,-0.064560,41.327623,0
4,ABBV,153.13,0.040090,53.961456,0.022247,66.381156,0.026917,92.505353,0
...,...,...,...,...,...,...,...,...,...
499,YUM,121.98,0.012371,48.608137,-0.074557,37.259101,-0.069152,37.259101,0
500,ZBH,110.76,-0.089179,23.768737,-0.144209,16.488223,-0.108712,14.132762,0
501,ZBRA,225.91,-0.171387,7.280514,-0.200885,8.137045,-0.149076,5.567452,0
502,ZION,33.17,0.203902,86.081370,-0.103603,27.194861,-0.077124,29.978587,0


In [134]:
from statistics import mean

for row in hqm_dataframe.index:
    row_percentiles=[]
    for time_period in time_periods:
        row_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])

    hqm_dataframe.loc[row, 'HQM Score'] = mean(row_percentiles)


hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,200-Day Price Return,200-Day Return Percentile,50-Day Price Return,50-Day Return Percentile,HQM Score
0,A,110.50,-0.148367,9.850107,-0.167640,12.633833,-0.081727,27.623126,16.702355
1,AAL,12.61,-0.128348,14.775161,-0.165346,13.062099,-0.159468,3.640257,10.492505
2,AAP,54.82,-0.356030,0.214133,-0.485734,0.214133,-0.178987,1.927195,0.785153
3,AAPL,170.43,0.116156,71.092077,0.027022,68.308351,-0.064560,41.327623,60.242684
4,ABBV,153.13,0.040090,53.961456,0.022247,66.381156,0.026917,92.505353,70.949322
...,...,...,...,...,...,...,...,...,...
499,YUM,121.98,0.012371,48.608137,-0.074557,37.259101,-0.069152,37.259101,41.042113
500,ZBH,110.76,-0.089179,23.768737,-0.144209,16.488223,-0.108712,14.132762,18.129907
501,ZBRA,225.91,-0.171387,7.280514,-0.200885,8.137045,-0.149076,5.567452,6.995004
502,ZION,33.17,0.203902,86.081370,-0.103603,27.194861,-0.077124,29.978587,47.751606


In [136]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)
hqm_dataframe

,index,Ticker,Price,One-Year Price Return,One-Year Return Percentile,200-Day Price Return,200-Day Return Percentile,50-Day Price Return,50-Day Return Percentile,HQM Score
0,194,FTI,21.29,0.810315,99.785867,0.401469,100.000000,0.111749,99.785867,99.857245
1,6,ABMD,381.02,0.365295,97.002141,0.306318,99.357602,0.084239,99.357602,98.572448
2,212,HAL,42.03,0.366606,97.216274,0.167803,97.644540,0.056811,97.430407,97.430407
3,283,LLY,549.76,0.345737,96.145610,0.312941,99.571734,0.040271,94.432548,96.716631
4,315,MPC,152.43,0.261458,91.862955,0.213012,99.143469,0.061816,98.286938,96.431121
5,22,AKAM,106.96,0.247388,90.364026,0.210359,98.929336,0.058057,97.644540,95.645967
6,411,SLB,60.80,0.375673,97.430407,0.152422,95.074946,0.034506,93.790150,95.431834
7,227,HRB,42.39,0.206276,86.509636,0.189437,98.501071,0.123384,100.000000,95.003569
8,383,PSX,120.80,0.243747,90.149893,0.167704,97.430407,0.053417,97.216274,94.932191
9,37,ANET,181.60,0.381567,97.858672,0.178018,98.286938,0.002300,85.653105,93.932905


In [137]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(posotion_size/hq,)